Cargamos los modulos que necesitamos.

In [ ]:
import numpy as np
from matplotlib import pyplot
import sklearn as skl
import sklearn.pipeline as sklpipilne
import sklearn.model_selection as sklms
import sklearn.decomposition as skldeco
import sklearn.neighbors as skln

import sys
sys.path.append('..\install\lib')
import mnpy as mn

import reader
import metnum

Ahora debemos cargar la data con la que vamos a trabajar.

In [ ]:
df = reader.read7Zip('../data/train.7z')
targets = df['label'].to_numpy()
# la primer columna es el label, el resto son los 28*28 pixeles de cada imagen
images = df.iloc[:, 1:].to_numpy()

cols = 10
rows = 4
samples = rows*cols
titles = ["Dígito %d" % digit for digit in targets[0:samples] ]
metnum.plot_gallery(images[0:samples], titles, 28, 28, rows, cols)
pyplot.show()


Configuramos el pipeline con PCA como estimator y knn como scorer

In [ ]:
training_dataset = images[0:10000]
training_targets = targets[0:10000]

# estimators = [("pca", mn.PCA()), ("knn", skln.KNeighborsClassifier())]
estimators = [("pca", mn.PCA()), ("knn", mn.kNNClassifier())]
pipeline = sklpipilne.Pipeline(estimators)

param_grid = [
    {
        # PCA parameters
        'pca__n_components': [24],
        'pca__iterated_power': [128],
        'pca__tolerance_error': [0.0001],

        # KNN parameters
        'knn__k_neighbors': [3, 4],
        'knn__distance_metric': [mn.DistanceMetric.Euclidean, mn.DistanceMetric.Manhattan, mn.DistanceMetric.Chebyshev],
        # 'knn__weights': [mn.Weights.Uniform, mn.Weights.Distance],
    },
]

# kfold = sklms.KFold(n_splits=10, shuffle=True)
kfold = sklms.KFold(n_splits=10)

grid_search = sklms.GridSearchCV(estimator=pipeline, cv=kfold, param_grid=param_grid, n_jobs=8)
grid_search.fit(X=training_dataset, y=training_targets)
print(grid_search.best_params_)


Momento de graficar

In [ ]:
def graph_config(legends, results):
    old_n_neighbors = legends[0]['knn__k_neighbors']
    set_of_lists_with_results = []
    list_with_results = []
    graph_names = []
    for index, legend in enumerate(legends):
        new_n_neighbors = legend['knn__k_neighbors']

        if new_n_neighbors == old_n_neighbors:
            list_with_results.append(results[index])
        else:
            graph_names.append(old_n_neighbors)
            set_of_lists_with_results.append(list_with_results)
            list_with_results = [results[index]]
            old_n_neighbors = new_n_neighbors
    graph_names.append(new_n_neighbors)
    set_of_lists_with_results.append(list_with_results)
    return graph_names, set_of_lists_with_results

In [ ]:
results = grid_search.cv_results_['mean_test_score']
legends = grid_search.cv_results_['params']

best_params = grid_search.best_params_
best_k_neighbors = best_params['knn__k_neighbors']
best_n_components = best_params['pca__n_components']

graph_names, set_of_lists_with_results = graph_config(legends, results)
#print(best_params)
# print(max(results))
print(legends)
print(results)

